My Hamiltonian  (assuming all qubits are perfect two level system)
$$H(t) = H_0 + H_{int}(t) + H_{d1}(t) + H_{d2}(t) $$
$$\Big[ -\frac{1}{2}\omega_{q1}(\Phi_1(t)) \hat{\sigma}_{z1} \Big ] +
\Big[-\frac{1}{2}\omega_{q2}(\Phi_2(t)) \hat{\sigma}_{z2} \Big] +
\Big[g(t)\hat\sigma_{y1}\hat\sigma_{y2}\Big] +
\Big[\Omega_1V_{d1}(t)\hat{\sigma}_{y1} \Big] + \Big[\Omega_1V_{d1}(t)\hat{\sigma}_{y2}\Big]$$
https://nbviewer.org/urls/qutip.org/qutip-tutorials/tutorials-v4/pulse-level-circuit-simulation/qip-customize-device.ipynb

In [32]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import basis, fidelity, sigmax, sigmay, sigmaz, tensor, identity
from qutip.qip.circuit import QubitCircuit
from qutip.qip.compiler import GateCompiler, Instruction, SpinChainCompiler
from qutip.qip.device import ModelProcessor, Processor
from qutip.qip.noise import Noise

In [11]:
#ㅇㅣ지랄 굳이 해야하나..
class MyProcessor(ModelProcessor):
    """
    Custom processor built using ModelProcessor as the base class.
    This custom processor will inherit all the methods of the base class
    such as setting up of the T1 and T2 decoherence rates in the simulations.

    In addition, it is possible to write your own functions to add control
    pulses.

    Args:
        num_qubits (int): Number of qubits in the processor.
        t1, t2 (float or list): The T1 and T2 decoherence rates for the
    """

    def __init__(self, num_qubits,, t1=None, t2=None):
        super(MyProcessor, self).__init__(
            num_qubits, t1=t1, t2=t2
        )  # call the parent class initializer
        # The control pulse is discrete or continous.
        self.pulse_mode = "discrete"
        self.model.params.update(
            {
                # can also be different for each qubit
                "s1": [h_z] * num_qubits,
                "s2": [h_x] * num_qubits,
                "g": [g] * num_qubits,
            }
        )
        # The dimension of each controllable quantum system
        self.model.dims = [2] * num_qubits
        self.num_qubits = num_qubits
        self.set_up_ops()  # set up the available Hamiltonians

    def set_up_ops(self):
        """
        Sets up the control operators.
        """
        for m in range(self.num_qubits):
            # sigmax pulse on m-th qubit with the corresponding pulse
            self.add_control(2 * np.pi * sigmax(), m, label="sx" + str(m))
        # sz
        for m in range(self.num_qubits):
            self.add_control(2 * np.pi * sigmaz(), m, label="sz" + str(m))
        # interaction operator
        operator = tensor([sigmay(), sigmay()])
        for m in range(self.num_qubits - 1):
            self.add_control(2 * np.pi * operator, [m, m + 1],
                             label="g" + str(m))

SyntaxError: invalid syntax (737981160.py, line 15)

# What's the difference?

In [33]:
num_qubits = 2
omega1 = 0.01 * 2 * np.pi #10MHz
omega2 = 0.01 * 2 * np.pi #10MHz
omegaq1 = 4 #4GHz
omegaq2 = 4.3 #4.3GHz
v1 = 1.
v2 = 1.

target=0 을 설정하면 tensor([identity(),sigmay()]) 와 같은 효과를 얻게 되나???
ㄴㄴ target은 그냥 설정해줘야하는 값임 
위에 저렇게 tensor product 해주면 target=[0,1] 둘 다 

In [40]:
processor = Processor(N=2, spline_kind='step_func')
processor.add_drift((-0.5)*omegaq1*sigmaz(), targets=0)
processor.add_drift((-0.5)*omegaq2*sigmaz(), targets=1)
processor.add_control(tensor([sigmay(), sigmay()]),label="g",targets=[0,1])
processor.add_control(omega1*v1*sigmay(),label="s1",targets=0)
processor.add_control(omega2*v2*sigmay(),label="s2",targets=1)


In [31]:
qc = QubitCircuit(N=2)
qc.add_gate("X", 0)
qc.add_gate("ISWAP", targets=[0, 1])

NotImplementedError: Use the function in the sub-class